# **Spam Email Detection — Naive Bayes (Machine Learning)**

This notebook contains a simple, reproducible implementation of an email spam detection pipeline using the Naive Bayes classifier. The project uses the "Spam Email Classification" dataset from Kaggle and demonstrates data loading, preprocessing, model training, evaluation, and basic model export.

## **Step 00** : Install nessessary packages

In [13]:
! pip install numpy pandas

## **Step 01** : Data loading and cleaning

In [14]:
import numpy as np
import pandas as pd

# Load Data from CSV file
df = pd.read_csv("data/email.csv")

# Get Some info about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [15]:
# print the first and last 5 values to ch
print(df.head(5))
print(df.tail(5))

  Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
     Category                                            Message
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name


In [16]:
# Change the category to a binary values (0 or 1) based on the message is spam (1) or not spam (0)
df.loc[df["Category"] == "ham"] = 0
df.loc[df["Category"] == "spam"] = 1


# print the first and last 5 values to ch
print(df.head(5))
print(df.tail(5))

  Category Message
0        0       0
1        0       0
2        1       1
3        0       0
4        0       0
     Category Message
5567        1       1
5568        0       0
5569        0       0
5570        0       0
5571        0       0


**And that it for the data manipulation we need for now !!**